# Predefined Cluster Sequences
Example demonstrating how to use predefined cluster assignments and centers.

This is useful when you want to transfer clustering results from one time series to another.

Author: Maximilian Hoffmann

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd

# Configure Plotly for sphinx/nbsphinx output
import plotly.io as pio

import tsam
from tsam import ClusterConfig

pio.renderers.default = "notebook"

# Ensure results directory exists
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

Show a slice of the dataset

In [ ]:
raw.head()

Show the shape of the raw input data: 4 types of timeseries (GHI, Temperature, Wind and Load) for every hour in a year

In [ ]:
raw.shape

Plot an example series - in this case the wind speed

In [ ]:
# Original wind heatmap
tsam.plot.heatmap(raw, column="Wind", period_duration=24, title="Original Wind")

### Hierarchical aggregation

Initialize an aggregation class object with hierarchical clustering as method for eight typical days, without any integration of extreme periods. Alternative clusterMethod's are 'averaging','hierarchical' and 'k_medoids'.

In [ ]:
result = tsam.aggregate(
    raw,
    n_clusters=8,
    period_duration=24,
    cluster=ClusterConfig(method="hierarchical", representation="medoid"),
)

Create the typical periods

In [ ]:
cluster_representatives = result.cluster_representatives

In [ ]:
cluster_representatives

Show shape of typical periods: 4 types of timeseries for 8*24 hours

In [ ]:
cluster_representatives.shape

Repredict the original time series based on the typical periods

In [ ]:
reconstructed = result.reconstruct()

Plot the repredicted data

In [ ]:
# Predicted wind heatmap (all attributes)
tsam.plot.heatmap(
    reconstructed,
    column="Wind",
    period_duration=24,
    title="Predicted Wind (All Attributes)",
)

### Now cluster the wind time series only

Clustering the solar time series only with 8 typical days and hierarchical clustering leads to different typical days in another sequence.

Isolate wind time series and show first lines of data

In [ ]:
raw_wind = raw.loc[:, "Wind"].to_frame()
raw_wind.head()

Now same clustering procedure as above for the isolated wind time series

In [ ]:
result_wind = tsam.aggregate(
    raw_wind,
    n_clusters=8,
    period_duration=24,
    cluster=ClusterConfig(method="hierarchical", representation="medoid"),
)

In [ ]:
cluster_representatives_wind = result_wind.cluster_representatives

Export for preprocess time series for testing

In [ ]:
# Export preprocessed time series for testing/debugging.
# Note: The _aggregation attribute is internal and may change in future versions.
# This access pattern is appropriate for testing but should not be relied upon in production code.
result_wind._aggregation.normalizedPeriodlyProfiles.to_csv(
    RESULTS_DIR / "preprocessed_wind.csv"
)

In [ ]:
cluster_representatives_wind.shape

In [ ]:
reconstructed_wind = result_wind.reconstruct()

In [ ]:
# Predicted wind heatmap (wind only)
tsam.plot.heatmap(
    reconstructed_wind,
    column="Wind",
    period_duration=24,
    title="Predicted Wind (Wind Only)",
)

When we compare both plots, we see that 8 typical periods for wind only can better account extreme periods, but the cluster order in general changes

In [ ]:
result.cluster_assignments

In [ ]:
result_wind.cluster_assignments

### Predefining cluster sequence

tsam offers the option to aggregate input time series for a predefined cluster order. This means that we can take the cluster Order from the wind time series only and set it as input for the aggregation process for all attributes

In [ ]:
# Use predefined cluster assignments from the wind-only aggregation
result_predef = tsam.aggregate(
    raw,
    n_clusters=8,
    period_duration=24,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="medoid",
        predef_cluster_assignments=tuple(result_wind.cluster_assignments),
    ),
)

In [ ]:
cluster_representatives_predef = result_predef.cluster_representatives

In [ ]:
cluster_representatives_predef.shape

Save typical periods to .csv file

In [ ]:
cluster_representatives_predef.to_csv(
    RESULTS_DIR / "testperiods_predef_cluster_order.csv"
)

In [ ]:
reconstructed_predef = result_predef.reconstruct()

Now we compare the cluster orders

In [ ]:
result_wind.cluster_assignments

In [ ]:
result_predef.cluster_assignments

As it can be seen, the cluster order for the four attributes (i.e. he sequence of typical days) is no identical to the cluster order of the wind time series clustering. Now the color plots can be compared:

In [ ]:
# Predicted wind heatmap (predefined cluster order)
tsam.plot.heatmap(
    reconstructed_predef,
    column="Wind",
    period_duration=24,
    title="Predicted Wind (Predefined Cluster Order)",
)

As it can be seen, the plot for the aggregated wind time series only and the one for four with the predefined cluster Order from the wind time series still differ from each other. This is because of the fact, that only the cluster Order, but not the cluster centers of each cluster are predefined. Since these are in one case deterined for the wind time series only and in the other case for all four attributes in common, the chosen cluster centers (chosen typical days) differ from each other

### Predefining cluster order and cluster centers

If the cluster order and the cluster centers should be taken from the wind time series clustering, we pass the information which days where chosen as typical days for the wind time series to the aggregation of all four typical attributes as well

In [ ]:
# Use predefined cluster assignments AND cluster centers from wind-only aggregation
result_predef_with_centers = tsam.aggregate(
    raw,
    n_clusters=8,
    period_duration=24,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="medoid",
        predef_cluster_assignments=tuple(result_wind.cluster_assignments),
        predef_cluster_centers=tuple(result_wind.cluster_centers),
    ),
)

In [ ]:
cluster_representatives_predef_with_centers = (
    result_predef_with_centers.cluster_representatives
)

In [ ]:
cluster_representatives_predef_with_centers.shape

Save typical periods to .csv file

In [ ]:
cluster_representatives_predef_with_centers.to_csv(
    RESULTS_DIR / "testperiods_predef_order_and_centers.csv"
)

In [ ]:
reconstructed_predef_with_centers = result_predef_with_centers.reconstruct()

In [ ]:
# Predicted wind heatmap (predefined cluster order and centers)
tsam.plot.heatmap(
    reconstructed_predef_with_centers,
    column="Wind",
    period_duration=24,
    title="Predicted Wind (Predefined Order & Centers)",
)

Now even the chosen typical days for the four attributes are the same as for the aggregated wind time series only

### Simplified Transfer with `predefined`

The simplest way to transfer all assignments (cluster order, cluster centers, and segments) is to use the `predefined` parameter with `result.predefined`.

In [ ]:
from tsam import PredefinedConfig, SegmentConfig

First, aggregate the wind time series with segmentation:

In [ ]:
# Aggregate wind with segmentation
result_wind_seg = tsam.aggregate(
    raw_wind,
    n_clusters=8,
    period_duration=24,
    cluster=ClusterConfig(method="hierarchical", representation="medoid"),
    segments=SegmentConfig(n_segments=6),
)

Get the `predefined` config which contains all transfer state:

In [ ]:
# The predefined property contains all transfer state
result_wind_seg.predefined

In [ ]:
# Can be serialized to JSON
result_wind_seg.predefined.to_dict()

Transfer to all attributes with a single `predefined` parameter:

In [ ]:
# Transfer everything with a single predefined parameter
result_full_transfer = tsam.aggregate(
    raw,
    n_clusters=8,
    period_duration=24,
    predefined=result_wind_seg.predefined,
)

In [ ]:
# The result has the same segmented structure
result_full_transfer.cluster_representatives

### Saving and Loading predefined

The `predefined` can be saved to JSON and loaded later:

In [ ]:
import json

# Save to JSON
with open(RESULTS_DIR / "predefined.json", "w") as f:
    json.dump(result_wind_seg.predefined.to_dict(), f, indent=2)

# Load from JSON
with open(RESULTS_DIR / "predefined.json") as f:
    predef_loaded = PredefinedConfig.from_dict(json.load(f))

# Use loaded predefined
result_from_file = tsam.aggregate(raw, n_clusters=8, predefined=predef_loaded)
result_from_file.cluster_representatives.head()

### Unified Assignments DataFrame

The `assignments` property provides a convenient DataFrame with all assignment information for each original timestep:

In [ ]:
# Unified assignments DataFrame
result_full_transfer.assignments.head(30)